In [133]:
import json
import warnings
import numpy as np
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
warnings.simplefilter("ignore")

# reduced fool-proof
UNKNOWN = ['n/a', 'N/A', 'na', 'NA', '', np.nan, 'unknown', 'UNKNOWN', '?', '??', "???"]
YES = [1, 'yes', 'y', 'Yes', 'YES', 'Y']
NO = [0, 'no', 'n', 'No', 'NO', 'N']

In [126]:
"""
Author: Haining Wang hw56@indiana.edu
"""

import warnings
import numpy as np
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
warnings.simplefilter("ignore")

# reduced fool-proof
UNKNOWN = ['n/a', 'N/A', 'na', 'NA', '', np.nan, 'unknown', 'UNKNOWN', '?', '??', "???"]
YES = [1, 'yes', 'y', 'Yes', 'YES', 'Y']
NO = [0, 'no', 'n', 'No', 'NO', 'N']


class DTA(object):
    """
    DTA class is used to do the heavy lifting on https://visual-dta.herokuapp.com/.
    Generates a graph with read-in dataframe.
    """

    def __init__(self, df):
        # construct a dataframe to hold user input
        self.df = df
        # construct a graph
        self.graph = nx.Graph()
        # hold nodes' position
        self.nodes = {}
        # edges
        # self.edges = []
        # self.edges_dotted = []
        self.edge_x = []
        self.edge_y = []
        self.edge_dotted_x = []
        self.edge_dotted_y = []


    def process_nodes(self):
        # calculates x, y coordinates from data
        node_x = []
        node_y = []

        for idx, row in self.df.iterrows():
            # root, root should starts from x==0, suggested by SCH
            if idx == 0:
                cur_x = 0.0
                cur_y = float(self.df.Proposition.count())
            # tree
            else:
                # deal with the P,T, B situation
                # cur_x' calculation is subjective to position (whether is the first point) and
                # 'Responds To' value
                # TODO: ROBUST ('Responds To')
#                 if not row['Responds To'] in UNKNOWN:
                try:
                    # for propositions coded with pure number
                    cur_x = self.nodes[int(row['Responds To'])][0] + float(row.Distance)
                except:
                    # in case of string proposition
                    cur_x = self.nodes[row['Responds To']][0] + float(row.Distance)
#                 else:
#                     # deal with the [B]reak situation
#                     # conveniently set the current point four units to the root
# #                     cur_x = self.nodes[0][0] + 4
# #                     cur_x += 4
#                     pass
                cur_y = float(self.df.Proposition.count() - idx)

            node_x.append(cur_x)
            node_y.append(cur_y)
            self.nodes.update({row.Proposition: [cur_x, cur_y]})

        for node in self.df.Proposition:
            self.graph.add_node(node, pos=(self.nodes[node]))

        # for (node, xy) in zip(self.nodes.keys(), list(zip(node_x, node_y))):
        #     graph.add_node(node, pos=(xy[0], xy[1]))

        # use df to trace positions
        self.df['Pos'] = pd.Series(list(self.nodes.values()))


    def process_edges(self):
        """
        TODO: FUNCTION: how to tell dotted lines apart from normal lines
        Finds edges and dotted edges
        :return:
        """
        for idx1, row1 in self.df.iterrows():
            for idx2, row2 in self.df.iterrows():
                if row1.Proposition == row2['Responds To'] and row2['Dotted Line'] in NO:
                    # self.edges.append([row1.Proposition, row2.Proposition])
                    self.graph.add_edge(row1.Proposition, row2.Proposition, dotted=False)
                #                 edges.append((row1.Proposition, row2.Proposition))
                elif row1.Proposition == row2['Responds To'] and row2['Dotted Line'] in YES:
                    # self.edges_dotted.append([row1.Proposition, row2.Proposition])
                    self.graph.add_edge(row1.Proposition, row2.Proposition, dotted=True)
                #                 edges_dotted.append((row1.Proposition, row2.Proposition))
                else:
                    # TODO: ROBUST
                    # should raise some error to guide users
                    pass

        for edgeTuple_dottedDict in list(self.graph.edges.data()):
            x0, y0 = self.graph.nodes[edgeTuple_dottedDict[:2][0]]['pos']
            x1, y1 = self.graph.nodes[edgeTuple_dottedDict[:2][1]]['pos']
            if not edgeTuple_dottedDict[2]['dotted']:
                self.edge_x.append(x0)
                self.edge_x.append(x1)
                self.edge_x.append(None)
                self.edge_y.append(y0)
                self.edge_y.append(y1)
                self.edge_y.append(None)
            else:
                self.edge_dotted_x.append(x0)
                self.edge_dotted_x.append(x1)
                self.edge_dotted_x.append(None)
                self.edge_dotted_y.append(y0)
                self.edge_dotted_y.append(y1)
                self.edge_dotted_y.append(None)
            # x0, y0 = self.graph.nodes[edge[0]]['pos']
            # x1, y1 = self.graph.nodes[edge[1]]['pos']
            # self.edge_x.append(x0)
            # self.edge_x.append(x1)
            # self.edge_x.append(None)
            # self.edge_y.append(y0)
            # self.edge_y.append(y1)
            # self.edge_y.append(None)

    def node_trace_t(self):
        """

        :return:
        """
        node_trace_t = go.Scatter(
            # first entry + all T
            x=[self.df.iloc[0:1].Pos[0][0]] + [x for [x, y] in self.df[self.df['Relation Type'] == 'T'].Pos],
            y=[self.df.iloc[0:1].Pos[0][1]] + [y for [x, y] in self.df[self.df['Relation Type'] == 'T'].Pos],
            name='Narrowly on-topic (T)',
            mode='markers',
            hoverinfo='text',
            #     hovertext=annotations,
            marker=dict(
                showscale=False,
                #         colorscale='Viridis',
                #         reversescale=True,
                color='blue',
                size=8,
                line_width=2))
        return node_trace_t

    def node_trace_p(self):
        """

        :return:
        """
        node_trace_p = go.Scatter(
            # first entry + all T
            x=[self.df.iloc[0:1].Pos[0][0]] + [x for [x, y] in self.df[self.df['Relation Type'] == 'P'].Pos],
            y=[self.df.iloc[0:1].Pos[0][1]] + [y for [x, y] in self.df[self.df['Relation Type'] == 'P'].Pos],
            name='Parallel Shift (P)',
            mode='markers',
            hoverinfo='text',
            #     hovertext=annotations,
            marker=dict(
                showscale=False,
                #         colorscale='Viridis',
                #         reversescale=True,
                color='red',
                size=8,
                line_width=2))
        return node_trace_p

    def node_trace_b(self):
        """

        :return:
        """
        node_trace_b = go.Scatter(
            # first entry + all T
            x=[self.df.iloc[0:1].Pos[0][0]] + [x for [x, y] in self.df[self.df['Relation Type'] == 'B'].Pos],
            y=[self.df.iloc[0:1].Pos[0][1]] + [y for [x, y] in self.df[self.df['Relation Type'] == 'B'].Pos],
            name='Break (B)',
            mode='markers',
            hoverinfo='text',
            #     hovertext=annotations,
            marker=dict(
                showscale=False,
                #         colorscale='Viridis',
                #         reversescale=True,
                color='green',
                size=8,
                line_width=2))
        return node_trace_b

    def edge_trace(self):
        """

        :return:
        """
        edge_trace = go.Scatter(
            x=self.edge_x,
            y=self.edge_y,
            name="responds to",
            line=dict(width=1, color='#888'),
            hoverinfo='none',
            mode='lines')
        return edge_trace

    def edge_dotted_trace(self):
        """

        :return:
        """
        edge_dotted_trace = go.Scatter(
            x=self.edge_dotted_x,
            y=self.edge_dotted_y,
            name="may respond to",
            line=dict(dash='dot', width=1, color='#888'),
            hoverinfo='none',
            mode='lines')
        return edge_dotted_trace

    def annotations(self):
        annotations = [dict(
            x=self.df.iloc[i].Pos[0],
            y=self.df.iloc[i].Pos[1],
            hovertext=self.df.iloc[i].Speaker,
            xanchor='left',
            yanchor='bottom',
            text=self.df.iloc[i].Text,
            visible=True,
            showarrow=False)
            for i in range(self.df.shape[0])]

        return annotations
    
    def no_annotations(self):
        annotations = [dict(
            x=self.df.iloc[i].Pos[0],
            y=self.df.iloc[i].Pos[1],
            hovertext=self.df.iloc[i].Speaker,
            xanchor='left',
            yanchor='bottom',
            text=[],
            visible=True,
            showarrow=False)
            for i in range(self.df.shape[0])]

        return annotations

    def draw_graph(self, 
#               retrun=None
             ):
        fig = go.Figure(data=[self.node_trace_t(), self.node_trace_p(), self.node_trace_b(),
                              self.edge_trace(), self.edge_dotted_trace()],
                        layout=go.Layout(
                            # title='Visual-DTA: A Demo',
                            titlefont_size=20,
                            showlegend=True,
                            hovermode='closest',
                            width=1000,
                            height=1000,
                            margin=dict(b=20, l=5, r=5, t=40),
                            annotations=self.annotations(),
                            xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
                            yaxis=dict(showgrid=False, zeroline=False, showticklabels=True))
                        )
        # fig.show()
        
#         fig.updatemenus = [{}, {}, {}]
        
#         fig.update_layout(
#             updatemenus=[
#                 dict(
#                     type="buttons",
#                     direction="right",
#                     active=0,
#                     x=0.57,
#                     y=1.2,
#                     buttons=list([
#                         dict(label="Show Text",
#                              method="restyle",
#                              args=[{"annotations": self.annotations}]
#                             ),
#                         dict(label="Hide Text",
#                              method="restyle",
#                              args=[{"annotations": []}]
#                             )
#                     ]),
#                 )
#             ])

        
        self.fig = fig
        return fig


In [120]:
df = pd.read_excel('./samples/BiliBili_comments.xlsx')

In [127]:
dta = DTA(df)

In [128]:
dta.process_nodes()

In [81]:
# dta.nodes

{0: [0.0, 52.0],
 1: [1.0, 51.0],
 2: [1.0, 50.0],
 3: [1.0, 49.0],
 4: [1.0, 48.0],
 5: [0.0, 47.0],
 6: [4.0, 46.0],
 7: [2.0, 45.0],
 's34': [2.0, 44.0],
 9: [3.0, 43.0],
 10: [2.0, 42.0],
 11: [0.0, 41.0],
 12: [2.0, 40.0],
 13: [0.0, 39.0],
 14: [0.0, 38.0],
 15: [0.0, 37.0],
 16: [0.0, 36.0],
 17: [0.0, 35.0],
 18: [0.0, 34.0],
 19: [0.0, 33.0],
 20: [2.0, 32.0],
 'h1': [2.0, 31.0],
 22: [0.0, 30.0],
 23: [0.0, 29.0],
 24: [0.0, 28.0],
 25: [0.0, 27.0],
 26: [5.0, 26.0],
 27: [0.0, 25.0],
 30: [1.0, 24.0],
 31: [1.0, 23.0],
 32: [0.0, 22.0],
 33: [0.0, 21.0],
 34: [1.0, 20.0],
 35: [2.0, 19.0],
 36: [4.0, 18.0],
 37: [9.0, 17.0],
 38: [1.0, 16.0],
 39: [5.0, 15.0],
 40: [1.0, 14.0],
 41: [1.0, 13.0],
 42: [1.0, 12.0],
 43: [0.0, 11.0],
 44: [3.0, 10.0],
 45: [0.0, 9.0],
 46: [1.0, 8.0],
 47: [2.0, 7.0],
 48: [3.0, 6.0],
 49: [2.0, 5.0],
 50: [2.0, 4.0],
 51: [1.0, 3.0],
 52: [2.0, 2.0],
 53: [0.0, 1.0]}

In [129]:
dta.process_edges()

In [130]:
dta.draw_graph()

In [138]:
json.dumps(dta.annotations(), indent=2)

'[\n  {\n    "x": 0.0,\n    "y": 52.0,\n    "hovertext": "\\u975e\\u5b98\\u65b9\\u5e72\\u6270",\n    "xanchor": "left",\n    "yanchor": "bottom",\n    "text": "AKA.\\u897f\\u5929\\u56e2",\n    "visible": true,\n    "showarrow": false\n  },\n  {\n    "x": 1.0,\n    "y": 51.0,\n    "hovertext": "\\u545c\\u547c\\u5566\\u547c\\u82cf\\u6ea2\\u51fa",\n    "xanchor": "left",\n    "yanchor": "bottom",\n    "text": "\\u4f60\\u7684\\u5934\\u50cf\\u6211\\u5f88\\u559c\\u6b22[doge]\\u6240\\u4ee5\\u4f60\\u80fd\\u4e0d\\u80fd\\u628a\\u4f60\\u7684\\u5934\\u50cf\\u6362\\u4e86\\u8ba9\\u6211\\u6362\\u4e0a",\n    "visible": true,\n    "showarrow": false\n  },\n  {\n    "x": 1.0,\n    "y": 50.0,\n    "hovertext": "\\u975e\\u5b98\\u65b9\\u5e72\\u6270",\n    "xanchor": "left",\n    "yanchor": "bottom",\n    "text": "\\u80fd\\u628a\\u4f60\\u8001\\u5a46\\u9001\\u6211\\u5417\\uff1f\\u9001\\u4e86\\u5c31\\u6362",\n    "visible": true,\n    "showarrow": false\n  },\n  {\n    "x": 1.0,\n    "y": 49.0,\n    "hovertex

In [135]:
json.dumps(dta.fig, indent=2)

TypeError: Object of type Figure is not JSON serializable

In [117]:
import plotly.graph_objects as go

import pandas as pd

# load dataset
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/volcano.csv")

# create figure
fig = go.Figure()

# Add surface trace
fig.add_trace(go.Surface(z=df.values.tolist(), colorscale="Viridis"))

# Update plot sizing
fig.update_layout(
    width=800,
    height=900,
    autosize=False,
    margin=dict(t=0, b=0, l=0, r=0),
    template="plotly_white",
)

# Update 3D scene options
fig.update_scenes(
    aspectratio=dict(x=1, y=1, z=0.7),
    aspectmode="manual"
)

# Add dropdown
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            buttons=list([
                dict(
                    args=["type", "surface"],
                    label="3D Surface",
                    method="restyle"
                ),
                dict(
                    args=["type", "heatmap"],
                    label="Heatmap",
                    method="restyle"
                )
            ]),
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.11,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

# Add annotation
fig.update_layout(
    annotations=[
        dict(text="Trace type:", showarrow=False,
                             x=0, y=1.08, yref="paper", align="left")
    ]
)

fig.show()